In [1]:
import pandas as pd
import torch
from GeoDS import eda
from GeoDS import hypercube
from GeoDS import datawrangle
from GeoDS import utilities
from GeoDS.featureengineering import feature_extraction as feat_ext

import os
import glob
import matplotlib as plt
plt.rcParamsDefault["figure.figsize"] = [20., 20.]
plt.rcParams["figure.facecolor"] = 'white'
plt.rcParams["axes.facecolor"] = 'white'
plt.rcParams["savefig.facecolor"] = 'white'
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

2022-09-01 19:05:35.567941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-01 19:05:35.567993: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
trial_name = 'Mosaic_Minerals_3D'

output_folder = os.path.join(trial_name, 'outputs/')
    
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
df = pd.read_csv('new_MM_Gaboury_MAGINV_finescale12-5m.csv')
df.head()

,susc,X,Y,Z
0,-0.026063,639225.0,5241287.5,-700.0
1,-0.025617,639225.0,5241287.5,-687.5
2,-0.025171,639225.0,5241287.5,-675.0
3,-0.024681,639225.0,5241287.5,-662.5
4,-0.024279,639225.0,5241287.5,-650.0


In [4]:
df.shape

(34032948, 4)

In [5]:
#from GeoDS.featureengineering import feature_extraction as feat_ext

#stat_df = feat_ext.moving_window_3d(input_csv='new_MM_Gaboury_MAGINV_finescale12-5m.csv',
#                           output_folder = os.path.join(output_folder),
#                           col_name='susc',
#                           win_size=25, 
#                           stats = ['max', 'min','mean'])

In [6]:
import warnings
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from osgeo import gdal
import subprocess
import os
import rioxarray as rio
from scipy.spatial import distance_matrix
from GeoDS import utilities
import itertools
import pandas as pd
import geopandas
import rasterio
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
import math
from libpysal.weights import Queen, Rook, KNN, DistanceBand
from esda.getisord import G_Local
from esda.moran import Moran_Local
import rioxarray
import xrspatial
import xarray
from dask.dataframe import from_pandas

def moving_window_3d(input_data, output_folder, col_name, win_size, stats=[], save_csv = True, save_dask = False):
    """
    xarray rolling function works on dataset and data array and it has an in-built rolling function that generates sliding windows for n-dims.
    This function creates 6 1st order statistics for input passed as a CSV.
    This function is tested for Window_size of 3 and 5 on global energy 3d dataset.
    DF.shape is (23226192, 1) and wall time taken for window size 3 is <2 mins for calcualting all 6 statistics.
    However as we increase the window size, computation time increases abnormally to 30-40mins. I would urge caution to users using this function with very large window size along with variance and standard deviation calculations.
    Maximum time is spent while computing variance and standard deviation ,if you want you can remove those entires from stats list in the input arg"
    Example:
    >>> import pandas as pd
    >>> import xarray
    >>> import numpy as np
    >>> df = pd.read_csv("Grid of Points.csv",usecols=['X','Y','Z',"Mag_Inversion"],low_memory=False)
    >>> stat_df = moving_window_3d(input_csv="Grid of Points.csv", col_name="Mag_Inversion",win_size=3, stats = ['max', 'min','mean'])
    """
    
    if save_csv & save_dask == True:
        raise ValueError(
            "Processing stopped.Saving cube in csv and dask is not useful. Please choose any one format of csv or dask dataframe.")
        
    if (type(input_data) == str):
        if (input_data.endswith('.csv')):
            df = pd.read_csv(input_data, usecols=["X", "Y", "Z", col_name], low_memory=False)
            df = df.set_index(['X', 'Y', 'Z'])
        elif (input_data.endswith('.tif')):
            print("Not implemented yet!")                        
    elif (type(input_data) == pd.DataFrame):
        df = input_data.copy()
        df = df.set_index(['X', 'Y', 'Z'])
    else:
        print("Type for input data should be a .csv file or .tif file or a pandas dataframe. " + str(
            type(input_data)) + "not supported")        
    
    xr = df.to_xarray()
    # create a rolling object of the xarray-dataarray
    xr_roll = xr.rolling(X=win_size, Y=win_size, Z=win_size, center=True)

    # CAUSE OF ERROR FOR COMMENTED OUT LINES:
    # Since the dimension of xr_roll.mean().data_vars[col_name].values.flatten().shape
    # is ALWAYS larger than the dimension of the original dataframe (df), the length of values does not match and
    # we get a ValueError once we try to write the new stat arrays into the same dataframe.
    # I fix this bug by converting xr_roll.mean().data_vars[col_name] object to dataframe 
    # and then store it in another dataframe called stat_df.
    # FYI, stat_df also includes coordinates as it is a conversion from xarray.
    
    stat_df = pd.DataFrame() # New DataFrame to store moving window features!
    
    if "mean" in stats:
        #df[col_name + "_mean"] = xr_roll.mean().data_vars[col_name].values.flatten()
        stat_df[col_name + "_mean"] = xr_roll.mean().data_vars[col_name].to_dataframe()
        print("mean done")
    if "max" in stats:
        #df[col_name + "_max"] = xr_roll.max().data_vars[col_name].values.flatten()
        stat_df[col_name + "_max"] = xr_roll.max().data_vars[col_name].to_dataframe()
        print("max done")
    if "min" in stats:
        #df[col_name + "_min"] = xr_roll.min().data_vars[col_name].values.flatten()
        stat_df[col_name + "_min"] = xr_roll.min().data_vars[col_name].to_dataframe()
        print("min done")
    if "var" in stats:
        #df[col_name + "_var"] = xr_roll.var().data_vars[col_name].values.flatten()
        stat_df[col_name + "_var"] = xr_roll.var().data_vars[col_name].to_dataframe()
        print("var done")
    if "std" in stats:
        #df[col_name + "_std"] = xr_roll.std().data_vars[col_name].values.flatten()
        stat_df[col_name + "_std"] = xr_roll.std().data_vars[col_name].to_dataframe()
        print("std done")
    if "sum" in stats:
        #df[col_name + "_sum"] = xr_roll.sum().data_vars[col_name].values.flatten()
        stat_df[col_name + "_sum"] = xr_roll.sum().data_vars[col_name].to_dataframe()
        print("sum done")
    
    df.reset_index(inplace=True)
    stat_df.reset_index(inplace=True)
    
    df = df.merge(stat_df, how='left', on=['X', 'Y', 'Z']) # Let's merge the results with the original dataframe
    df.dropna(inplace=True)
    
    if save_csv == True:
        print('Saving the csv...')
        df.to_csv(os.path.join(output_folder, "moving_window_stats_win_size_"+str(win_size)+".csv"), index=False) 
                
    if (save_dask == True):
        if type(df) == pd.core.frame.DataFrame:
            df = from_pandas(df, chunksize=100)
            df = df.repartition(partition_size='250MB')
        
        output_file_name = os.path.join(output_folder, "moving_window_stats_win_size_"+str(win_size)+"-*.csv")
        print('Saving the csv...')
        df.to_csv(output_file_name, index=False)
    
    return df

In [9]:
stat_df = moving_window_3d(input_data=df,
                           output_folder = os.path.join(output_folder),
                           col_name='susc',
                           win_size=5, 
                           stats = ['mean', 'max', 'min', 'var', 'std', 'sum'],
                           save_csv = True,
                           save_dask = False)
 

mean done
max done
min done
var done
std done
sum done
Saving the csv...


In [10]:
stat_df

,X,Y,Z,susc,susc_mean,susc_max,susc_min,susc_var,susc_std,susc_sum
28897,639250.0,5241312.5,-662.5,-0.024655,-0.024672,-0.023272,-0.026356,6.043909e-07,0.000777,-3.083941
28898,639250.0,5241312.5,-650.0,-0.024168,-0.024181,-0.022857,-0.025777,5.128404e-07,0.000716,-3.022678
28899,639250.0,5241312.5,-637.5,-0.023727,-0.023722,-0.022470,-0.025195,4.454603e-07,0.000667,-2.965273
28900,639250.0,5241312.5,-625.0,-0.023331,-0.023292,-0.022174,-0.024724,4.103926e-07,0.000641,-2.911443
28901,639250.0,5241312.5,-612.5,-0.022902,-0.022870,-0.021766,-0.024143,4.002495e-07,0.000633,-2.858726
...,...,...,...,...,...,...,...,...,...,...
33996387,648437.5,5246587.5,212.5,0.000641,0.000405,0.001556,-0.000270,1.866283e-07,0.000432,0.050580
33996388,648437.5,5246587.5,225.0,0.000454,0.000295,0.001556,-0.000254,1.684728e-07,0.000410,0.036851
33996389,648437.5,5246587.5,237.5,0.000180,0.000140,0.001435,-0.000782,1.691009e-07,0.000411,0.017484
33996390,648437.5,5246587.5,250.0,-0.000099,-0.000033,0.001230,-0.001567,1.954506e-07,0.000442,-0.004160
